In [4]:
import json
import pandas as pd
import os
from IPython.display import display
# d = json.load(open('data/mpd.slice.0-999.json'))
# tracks = pd.DataFrame(d['playlists'][0]['tracks'])

# create list of playlists
playlists = []
path = 'data'
count = 5
for filename in os.listdir(path):
    count -= 1
    if count < 0:
        break
    f = os.path.join(path, filename)
    if os.path.isfile(f):
        d = json.load(open(f))
        playlists.append(pd.DataFrame(d['playlists']))

playlists = pd.concat(playlists)
playlists = playlists.reset_index(drop=True)
print(playlists.shape)
#track0 = pd.DataFrame(playlists['tracks'][0]).reset_index(drop=True)

(5000, 12)


In [5]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials as cc
import pandas as pd

In [6]:
# api setup
#pip install spotipy


client_id = '898dd71dc932407e85921f0ac79f0127'
client_secret = '16d994f070064371beb8758f32d64180'
#https://developer.spotify.com/dashboard/applications/898dd71dc932407e85921f0ac79f0127
client_cc = cc(client_id=client_id,client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_cc)

In [9]:
import warnings
# from playlists extract tracks with corresponding features from spotipy
import time
import numpy as np

sleep_min = 2
sleep_max = 3
start_time = time.time()
request_count = 0
tracks_w_features = pd.DataFrame()
recommendations = pd.DataFrame()
for i in range(0,5):
    request_count+=1
    if request_count % 2 == 0:
        #print(str(request_count) + " playlists extracted from dataset")
        time.sleep(np.random.uniform(sleep_min, sleep_max))
        #print('Loop #: {}'.format(request_count))
        #print('Elapsed Time: {} seconds'.format(time.time() - start_time))
    # extract audio features using spotipy api (called sp with credentials)
    if len(playlists['tracks'][i]) < 50:
        tracks = pd.DataFrame(sp.audio_features(pd.DataFrame(playlists['tracks'][i]).reset_index(drop=True).track_uri))
        tracks_w_features = tracks_w_features.append(tracks)
display(tracks_w_features)
# remove useless-categorical columns, can use index to get song information later
input = tracks_w_features.drop(columns=['time_signature', 'duration_ms','analysis_url','track_href','uri','id','type']).copy()
display(input.iloc[0])
warnings.filterwarnings("ignore")
warnings.filterwarnings(action='once')

C:\Users\SiBuX\AppData\Local\Temp\ipykernel_10388\1534800647.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_w_features = tracks_w_features.append(tracks)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.8150,0.438,0,-14.522,0,0.0346,0.216000,0.000466,0.0787,0.5520,108.965,audio_features,2HHtWyy5CgaQbC7XSoOb0e,spotify:track:2HHtWyy5CgaQbC7XSoOb0e,https://api.spotify.com/v1/tracks/2HHtWyy5CgaQ...,https://api.spotify.com/v1/audio-analysis/2HHt...,243773,4
1,0.3430,0.975,1,-2.502,0,0.1350,0.000032,0.991000,0.0515,0.1330,119.883,audio_features,1MYYt7h6amcrauCOoso3Gx,spotify:track:1MYYt7h6amcrauCOoso3Gx,https://api.spotify.com/v1/tracks/1MYYt7h6amcr...,https://api.spotify.com/v1/audio-analysis/1MYY...,70294,4
2,0.4140,0.959,7,-4.299,0,0.0378,0.000023,0.933000,0.3680,0.5890,145.911,audio_features,3x2mJ2bjCIU70NrH49CtYR,spotify:track:3x2mJ2bjCIU70NrH49CtYR,https://api.spotify.com/v1/tracks/3x2mJ2bjCIU7...,https://api.spotify.com/v1/audio-analysis/3x2m...,65306,4
3,0.5220,0.205,2,-7.986,1,0.0376,0.053400,0.235000,0.0985,0.3530,103.868,audio_features,1Pm3fq1SC6lUlNVBGZi3Em,spotify:track:1Pm3fq1SC6lUlNVBGZi3Em,https://api.spotify.com/v1/tracks/1Pm3fq1SC6lU...,https://api.spotify.com/v1/audio-analysis/1Pm3...,108532,4
4,0.0849,0.956,2,-4.196,1,0.0805,0.000052,0.953000,0.3970,0.3090,185.397,audio_features,1NXTEkIeRL59NK61QuhYUl,spotify:track:1NXTEkIeRL59NK61QuhYUl,https://api.spotify.com/v1/tracks/1NXTEkIeRL59...,https://api.spotify.com/v1/audio-analysis/1NXT...,214269,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
40,0.2590,0.571,10,-7.267,1,0.0458,0.511000,0.000000,0.1380,0.0992,87.485,audio_features,3PlmR2sBI5WJqvLlFaRa5O,spotify:track:3PlmR2sBI5WJqvLlFaRa5O,https://api.spotify.com/v1/tracks/3PlmR2sBI5WJ...,https://api.spotify.com/v1/audio-analysis/3Plm...,350267,4
41,0.3750,0.127,0,-15.582,0,0.0940,0.901000,0.000000,0.2960,0.1020,86.656,audio_features,3Wakuaf7Bdr48SY1eSEEgc,spotify:track:3Wakuaf7Bdr48SY1eSEEgc,https://api.spotify.com/v1/tracks/3Wakuaf7Bdr4...,https://api.spotify.com/v1/audio-analysis/3Wak...,155307,3
42,0.7590,0.951,5,-4.659,0,0.0811,0.106000,0.000010,0.9540,0.7430,110.881,audio_features,7hZRgQiZEKf1e8KTEVIkQN,spotify:track:7hZRgQiZEKf1e8KTEVIkQN,https://api.spotify.com/v1/tracks/7hZRgQiZEKf1...,https://api.spotify.com/v1/audio-analysis/7hZR...,369120,4
43,0.6910,0.967,0,-5.849,1,0.1530,0.125000,0.000001,0.7290,0.2690,122.033,audio_features,3rDJ4H6XTEpt5otqx4h1Ce,spotify:track:3rDJ4H6XTEpt5otqx4h1Ce,https://api.spotify.com/v1/tracks/3rDJ4H6XTEpt...,https://api.spotify.com/v1/audio-analysis/3rDJ...,308373,4


danceability          0.815000
energy                0.438000
key                   0.000000
loudness            -14.522000
mode                  0.000000
speechiness           0.034600
acousticness          0.216000
instrumentalness      0.000466
liveness              0.078700
valence               0.552000
tempo               108.965000
Name: 0, dtype: float64

C:\Users\SiBuX\AppData\Local\Temp\ipykernel_10388\1101085946.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_w_features = tracks_w_features.append(tracks)
C:\Users\SiBuX\AppData\Local\Temp\ipykernel_10388\1101085946.py:22: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tracks_w_features = tracks_w_features.append(tracks)


,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.8150,0.4380,0,-14.522,0,0.0346,0.216000,0.000466,0.0787,0.552,108.965,audio_features,2HHtWyy5CgaQbC7XSoOb0e,spotify:track:2HHtWyy5CgaQbC7XSoOb0e,https://api.spotify.com/v1/tracks/2HHtWyy5CgaQ...,https://api.spotify.com/v1/audio-analysis/2HHt...,243773,4
1,0.3430,0.9750,1,-2.502,0,0.1350,0.000032,0.991000,0.0515,0.133,119.883,audio_features,1MYYt7h6amcrauCOoso3Gx,spotify:track:1MYYt7h6amcrauCOoso3Gx,https://api.spotify.com/v1/tracks/1MYYt7h6amcr...,https://api.spotify.com/v1/audio-analysis/1MYY...,70294,4
2,0.4140,0.9590,7,-4.299,0,0.0378,0.000023,0.933000,0.3680,0.589,145.911,audio_features,3x2mJ2bjCIU70NrH49CtYR,spotify:track:3x2mJ2bjCIU70NrH49CtYR,https://api.spotify.com/v1/tracks/3x2mJ2bjCIU7...,https://api.spotify.com/v1/audio-analysis/3x2m...,65306,4
3,0.5220,0.2050,2,-7.986,1,0.0376,0.053400,0.235000,0.0985,0.353,103.868,audio_features,1Pm3fq1SC6lUlNVBGZi3Em,spotify:track:1Pm3fq1SC6lUlNVBGZi3Em,https://api.spotify.com/v1/tracks/1Pm3fq1SC6lU...,https://api.spotify.com/v1/audio-analysis/1Pm3...,108532,4
4,0.0849,0.9560,2,-4.196,1,0.0805,0.000052,0.953000,0.3970,0.309,185.397,audio_features,1NXTEkIeRL59NK61QuhYUl,spotify:track:1NXTEkIeRL59NK61QuhYUl,https://api.spotify.com/v1/tracks/1NXTEkIeRL59...,https://api.spotify.com/v1/audio-analysis/1NXT...,214269,3
5,0.2830,0.9250,3,-4.700,1,0.1110,0.000748,0.933000,0.1660,0.368,99.450,audio_features,3RGlJJFkWEavxeRQr9ivAd,spotify:track:3RGlJJFkWEavxeRQr9ivAd,https://api.spotify.com/v1/tracks/3RGlJJFkWEav...,https://api.spotify.com/v1/audio-analysis/3RGl...,110219,4
6,0.4920,0.9700,4,-5.677,0,0.0554,0.000010,0.891000,0.1620,0.556,120.005,audio_features,0e9hR1vTrzlUvFH5PgA9rY,spotify:track:0e9hR1vTrzlUvFH5PgA9rY,https://api.spotify.com/v1/tracks/0e9hR1vTrzlU...,https://api.spotify.com/v1/audio-analysis/0e9h...,207520,4
7,0.4930,0.9690,2,-3.282,0,0.0463,0.201000,0.655000,0.2470,0.340,170.581,audio_features,7dkbEHIMLoeuG4zXGmzhEH,spotify:track:7dkbEHIMLoeuG4zXGmzhEH,https://api.spotify.com/v1/tracks/7dkbEHIMLoeu...,https://api.spotify.com/v1/audio-analysis/7dkb...,226000,4
8,0.5360,0.4580,2,-9.906,0,0.0385,0.893000,0.495000,0.1060,0.259,170.113,audio_features,7nZnCjIJ8JigdMqJ36yRRm,spotify:track:7nZnCjIJ8JigdMqJ36yRRm,https://api.spotify.com/v1/tracks/7nZnCjIJ8Jig...,https://api.spotify.com/v1/audio-analysis/7nZn...,104600,1
9,0.6380,0.4900,4,-9.382,0,0.4260,0.271000,0.000000,0.6640,0.511,173.552,audio_features,2TK2KSrzXD6W01qjXVjNGh,spotify:track:2TK2KSrzXD6W01qjXVjNGh,https://api.spotify.com/v1/tracks/2TK2KSrzXD6W...,https://api.spotify.com/v1/audio-analysis/2TK2...,318215,4


danceability          0.815000
energy                0.438000
key                   0.000000
loudness            -14.522000
mode                  0.000000
speechiness           0.034600
acousticness          0.216000
instrumentalness      0.000466
liveness              0.078700
valence               0.552000
tempo               108.965000
Name: 0, dtype: float64

In [10]:
from sklearn.metrics.pairwise import cosine_similarity
df = pd.DataFrame(cosine_similarity(input, dense_output=True))
# display(df)
# 3 by input size matrix
# columns as follows:
# song a, song b, songs a similarity with b
# song a, song c, song a similarity with c
# song a, song d, song a similarity with d
feature_space = []
for i,j in enumerate(df.values):
    for p in range(len(j)):
        feature_space.append([i,p,j[p]])
# display(feature_space)

# remove 1's in diagonals
feature_space_refined = []
for i in range(len(feature_space)):
    if feature_space[i][0]==feature_space[i][1]:
        feature_space_refined.append([feature_space[i][0],feature_space[i][1],0])
    else:
        feature_space_refined.append(feature_space[i])
display(feature_space_refined[0:5])
display(feature_space_refined[len(feature_space_refined)-5:len(feature_space_refined)])

[[0, 0, 0],
 [0, 1, 0.9936796854932826],
 [0, 2, 0.9935171800428835],
 [0, 3, 0.9982063481431684],
 [0, 4, 0.9938527585832201]]

[[55, 51, 0.9981098409002281],
 [55, 52, 0.9994277565402166],
 [55, 53, 0.9997914130744283],
 [55, 54, 0.9985756430442937],
 [55, 55, 0]]

In [11]:
import webbrowser
from pandas._libs.internals import defaultdict

dict = defaultdict(list)
for i in range(len(feature_space_refined)):
    dict[feature_space_refined[i][0]].append(feature_space_refined[i][2])
df_updated = pd.DataFrame(dict)
display(df_updated)

# find max similarity and sort using argmax
sorted = []
for i in range(len(df_updated)):
    sorted.append(np.argmax(df_updated[i]))
#display(sorted)

# get the highest similarity of index position
sorted_updated = []
for i in sorted:
    sorted_updated.append(sorted[i])

similar_songs = pd.DataFrame (sorted_updated, columns=['Similar Song'])

similarity_value = pd.DataFrame(round(df_updated.max(axis=1),12), columns=['Similarity Value'])

parsed_songs = pd.DataFrame(sorted, columns=['Parsed Song'])

cs_df = pd.concat([parsed_songs,similar_songs,similarity_value],axis=1)

display(cs_df.head(5))
display(cs_df.tail(5))

,0,1,2,3,4,5,6,7,8,9,...,46,47,48,49,50,51,52,53,54,55
0,0.000000,0.993680,0.993517,0.998206,0.993853,0.995794,0.995776,0.993502,0.997145,0.996641,...,0.998731,0.995813,0.999856,0.993828,0.997860,0.999195,0.995280,0.994211,0.997915,0.995395
1,0.993680,0.000000,0.999170,0.998293,0.999967,0.999360,0.999333,0.999978,0.999255,0.999285,...,0.997754,0.999692,0.994523,0.998867,0.998786,0.997299,0.999526,0.997103,0.994727,0.998039
2,0.993517,0.999170,0.000000,0.998403,0.999265,0.999625,0.999731,0.999286,0.998902,0.999356,...,0.997670,0.998732,0.994619,0.999936,0.998061,0.997189,0.999721,0.999254,0.996737,0.999508
3,0.998206,0.998293,0.998403,0.000000,0.998471,0.999450,0.999388,0.998262,0.999739,0.999678,...,0.999898,0.999176,0.998774,0.998548,0.999762,0.999759,0.999266,0.998143,0.998586,0.999024
4,0.993853,0.999967,0.999265,0.998471,0.000000,0.999477,0.999411,0.999974,0.999330,0.999392,...,0.997930,0.999744,0.994766,0.999039,0.998891,0.997429,0.999651,0.997325,0.995000,0.998266
5,0.995794,0.999360,0.999625,0.999450,0.999477,0.000000,0.999940,0.999365,0.999661,0.999832,...,0.999030,0.999462,0.996678,0.999611,0.999283,0.998607,0.999899,0.998831,0.997529,0.999521
6,0.995776,0.999333,0.999731,0.999388,0.999411,0.999940,0.000000,0.999361,0.999665,0.999876,...,0.998921,0.999368,0.996597,0.999685,0.999171,0.998614,0.999903,0.999047,0.997701,0.999577
7,0.993502,0.999978,0.999286,0.998262,0.999974,0.999365,0.999361,0.000000,0.999229,0.999315,...,0.997690,0.999657,0.994399,0.999020,0.998706,0.997212,0.999597,0.997304,0.994786,0.998168
8,0.997145,0.999255,0.998902,0.999739,0.999330,0.999661,0.999665,0.999229,0.000000,0.999908,...,0.999516,0.999750,0.997761,0.998856,0.999819,0.999344,0.999656,0.997948,0.997504,0.998848
9,0.996641,0.999285,0.999356,0.999678,0.999392,0.999832,0.999876,0.999315,0.999908,0.000000,...,0.999335,0.999623,0.997356,0.999356,0.999589,0.999115,0.999858,0.998613,0.997782,0.999316


,Parsed Song,Similar Song,Similarity Value
0,24,0,0.999968
1,7,1,0.999978
2,49,12,0.999936
3,44,3,0.999967
4,40,4,0.999984


,Parsed Song,Similar Song,Similarity Value
51,37,51,0.999937
52,20,36,0.999951
53,11,53,0.999947
54,31,54,0.999920
55,45,55,0.999955


In [12]:
import random
# get most similar songs and open in browser
random_pair = random.randint(0, similar_songs.shape[0])
print(random_pair)

track0 = tracks_w_features.iloc[int(cs_df.iloc[random_pair][0])]
track1 = tracks_w_features.iloc[int(cs_df.iloc[random_pair][1])]

track0 = track0.drop(columns=['time_signature', 'duration_ms','analysis_url','track_href','uri','id','type']).copy()
track1 = track1.drop(columns=['time_signature', 'duration_ms','analysis_url','track_href','uri','id','type']).copy()

track0_uri = sp.track(tracks_w_features.iloc[int(cs_df.iloc[random_pair][0])].uri)
track1_uri = sp.track(tracks_w_features.iloc[int(cs_df.iloc[random_pair][1])].uri)

table = {'Track 0': track0, 'Track 1': track1}
table = pd.DataFrame(table)
display(table)

track0_url = track0_uri['external_urls']['spotify']
track1_url = track1_uri['external_urls']['spotify']

webbrowser.open(track0_url)
webbrowser.open(track1_url)

33


,Track 0,Track 1
danceability,0.39,0.528
energy,0.649,0.921
key,2,2
loudness,-5.163,-7.64
mode,1,1
speechiness,0.0345,0.0524
acousticness,0.172,0.0174
instrumentalness,0.0,0.000186
liveness,0.216,0.0472
valence,0.144,0.298


True